In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
#from google.colab import files
import io
from sklearn.model_selection import GridSearchCV


In [2]:
train = pd.read_csv("train.csv", dtype={"Age": np.float64})
test = pd.read_csv("test.csv", dtype={"Age": np.float64})

train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train.keys()


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:

predictor = [
    "Age",
    "Fare",
    "SibSp",
    "Parch",
    "FamilySize",
    "male",
    "female",
    "pclass_1",
    "pclass_2",
    "pclass_3",
]


In [6]:
def correct_data(titanic_data):
    #titanic_data.Sex = titanic_data.Sex.replace(['male', 'female'], [0, 1])
    titanic_data.Embarked = titanic_data.Embarked.fillna("S")
    #titanic_data.Embarked = titanic_data.Embarked.replace(['C', 'S', 'Q'], [0, 1, 2])
    titanic_data.Age = titanic_data.Age.fillna(titanic_data.Age.median())

    # Trying to add FamilySize
    titanic_data['FamilySize'] = titanic_data['SibSp'] + \
        titanic_data['Parch'] + 1

    # add Male & Female one-hot-encoding
    sex_dummy = pd.get_dummies(titanic_data.Sex)
    titanic_data['male'] = sex_dummy.male
    titanic_data['female'] = sex_dummy.female

    # add Embarked one-hot-encoding
    embarked_dummy = pd.get_dummies(titanic_data.Embarked)
    titanic_data['embarked_C'] = embarked_dummy.C
    titanic_data['embarked_S'] = embarked_dummy.S
    titanic_data['embarked_Q'] = embarked_dummy.Q

    # add Pclass one-hot-encoding
    pclass_dummy = pd.get_dummies(titanic_data.Pclass)
    titanic_data['pclass_1'] = pclass_dummy[1]
    titanic_data['pclass_2'] = pclass_dummy[2]
    titanic_data['pclass_3'] = pclass_dummy[3]

    # Cabin
    titanic_data.Cabin = titanic_data.Cabin.fillna("N")
    titanic_data['CabinDeck'] = titanic_data.Cabin.str.slice(0, 1)
    cabindeck_dummy = pd.get_dummies(titanic_data.CabinDeck)

    titanic_data['CabinDeck_A'] = cabindeck_dummy.A
    titanic_data['CabinDeck_B'] = cabindeck_dummy.B
    titanic_data['CabinDeck_C'] = cabindeck_dummy.C
    titanic_data['CabinDeck_D'] = cabindeck_dummy.D
    titanic_data['CabinDeck_E'] = cabindeck_dummy.E
    titanic_data['CabinDeck_F'] = cabindeck_dummy.F
    titanic_data['CabinDeck_G'] = cabindeck_dummy.G
    titanic_data['CabinDeck_NaN'] = cabindeck_dummy.N

    titanic_data.Fare = titanic_data.Fare.fillna(titanic_data.Fare.median())

    return titanic_data


In [7]:
train.head().Sex


0      male
1    female
2    female
3    female
4      male
Name: Sex, dtype: object

In [8]:
def replace_sex_to_0_1(titanic_data):
    titanic_data.Sex = titanic_data.Sex.replace(['male', 'female'], [0, 1])
    return titanic_data


#replace_sex_to_0_1 (train)
train.head().Sex


0      male
1    female
2    female
3    female
4      male
Name: Sex, dtype: object

In [9]:
train[train.Embarked.isna()].head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [10]:
def fill_na_embarked_with_S(titanic_data):
    titanic_data.Embarked = titanic_data.Embarked.fillna("S")
    return titanic_data


#fill_na_embarked_with_S (train)
train[train.Embarked.isna()].head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [11]:
correct_train = correct_data(train).sample(frac=1)

trainX = correct_train[predictor].values
trainY = correct_train.Survived.values


aiot4 = MLPClassifier(activation='tanh', solver='lbfgs', max_iter=5000,
                      alpha=1e-4, hidden_layer_sizes=(15, 8), random_state=4)
aiot4.fit(trainX, trainY)
print("Test set score: {:.3f}".format(aiot4.score(trainX, trainY)))


Test set score: 0.872


c:\Users\bensb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [ ]:
 
for max_iter in [500, 1000, 2000]:
    for alpha in [0.0001, 0.00001, 0.000003]:
        for solver in ["lbfgs", "sgd", "adam"]:
            for activation in ["identity", "logistic", "tanh", "relu"]:
                for hidden_layer_sizes in [(15, 8),(10, 8),(10, 8,3)]:
                    aiot4 = MLPClassifier(activation=activation, solver=solver, max_iter=max_iter,
                                        alpha=alpha, hidden_layer_sizes=hidden_layer_sizes, random_state=4)
                    aiot4.fit(trainX, trainY)
                    print("Iterations :", max_iter,"alpha :", alpha,"solver :", solver,"activation", activation,"hidden_layer_sizes", hidden_layer_sizes)
                    print("Test set score: {:.3f}".format(aiot4.score(trainX, trainY)))
                     


In [13]:
# test model with manual data
result = aiot4.predict([[1, 0, 0, 0, 0, 0, 1, 0, 0, 1]])
result 

ValueError: Expected 2D array, got 1D array instead:
array=[1 0 0 0 0 0 1 0 0 1].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.